# Author: Jingtang Ma

# Environment Configuration

If using google colab, run cell below:

In [22]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [23]:
#!pip install geemap

  Using cached geemap-0.13.4-py2.py3-none-any.whl (2.0 MB)
     |████████████████████████████████| 3.3 MB 15.5 MB/s 
     |████████████████████████████████| 95 kB 4.1 MB/s 
     |████████████████████████████████| 98 kB 9.0 MB/s 
  Using cached bqplot-0.12.33-py2.py3-none-any.whl (1.2 MB)
     |████████████████████████████████| 1.3 MB 56.0 MB/s 
  Using cached geeadd-0.5.5-py3-none-any.whl (30 kB)
  Using cached ipyevents-2.0.1-py2.py3-none-any.whl (130 kB)
     |████████████████████████████████| 99 kB 9.2 MB/s 
  Using cached ee_extra-0.0.13.tar.gz (187 kB)
     |████████████████████████████████| 3.0 MB 48.3 MB/s 
     |████████████████████████████████| 8.7 MB 38.6 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 128 kB 57.3 MB/s 
     |████████████████████████████████| 428 kB 46.2 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 342 kB 54.5 MB/s 
     |████████████████████████████

opencv-python==4.5.5.64 \\
opencv-contrib-python==4.5.5.64 

In [24]:
!pip install rasterio
!pip uninstall opencv-python
!pip uninstall opencv-contrib-python
!pip install opencv-python==4.5.5.64 
!pip install opencv-contrib-python==4.5.5.64 

Found existing installation: opencv-python 4.5.5.64
Uninstalling opencv-python-4.5.5.64:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python-4.5.5.64.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5Core-39545cc7.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5Gui-ba0a2070.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5Test-c38a5234.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5Widgets-e69d94fb.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5XcbQpa-ca221f44.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libX11-xcb-69166bdf.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libXau-00ec42fe.so.6.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libavcodec-65fa80df.so.58.134.100
    /usr/local/lib/python3.7/d

  Using cached opencv_contrib_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (66.7 MB)


# Import all Neccessary Libraries

In [25]:
import os
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.plot import show
from tqdm import tqdm
from pathlib import Path
from skimage import exposure
from sklearn.preprocessing import RobustScaler, MinMaxScaler

from osgeo import gdal
from PIL import Image
import pandas as pd
from math import floor
import cv2
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['show', 'floor']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Data Preprocessing

## Convert hyperspectral images from airborne (356 channels) to RGB (3 channels)

In [26]:
def hyperspectral2rgb(path):
  p = list(Path(path).glob("*.tif"))

  # i = 0
  for f in tqdm(p, total=len(p)):
      f = str(f)
      with rasterio.open(f) as src:
          if "RGB" in os.path.basename(f):
              r, g, b = src.read([1, 2, 3])
          else:
              r, g, b = src.read([82, 46, 24])
          meta = src.meta.copy()

      h, w = r.shape
      meta["nodata"] = 0
      meta["count"] = 3
      meta["driver"] = "GTiff"
      meta["dtype"] = rasterio.float32
      dest = f"{str(f).replace('.tif', '').strip('_RGB')}_RGB.tif"
      
      rgb = np.stack([r, g, b], axis=2)
      rgb[rgb < 0] = 0
      with rasterio.open(dest, "w", **meta) as dest_f:
          dest_f.write(rgb.transpose(2, 0, 1), [1, 2, 3])
          dest_f.set_band_description(1, "Red")
          dest_f.set_band_description(2, "Green")
          dest_f.set_band_description(3, "Blue")


      mask_nodata = rgb <= 0
      rgb[mask_nodata] = np.nan

      p1 = np.nanpercentile(rgb, 1.5, axis=(0, 1))
      p99 = np.nanpercentile(rgb, 98.5, axis=(0, 1))

      rgb = np.clip(rgb, p1, p99)
      rgb = (rgb - np.nanmin(rgb, axis=(0, 1))) / (np.nanmax(rgb, axis=(0, 1)) - np.nanmin(rgb, axis=(0, 1)))
      rgb = rgb.astype(np.float32)

      rgb[mask_nodata] = 1
      blue_mask = np.logical_and(rgb[:, :, 2] > 0.95, ~mask_nodata[:, :, 0])
      blue_data = rgb[blue_mask, :].copy() 
      rgb[blue_mask, :] = 0

      fig = plt.figure()
      plt.imshow(rgb)
      # plt.title(os.path.basename(str(f)))
      plt.axis('off')
      plt.savefig(dest.replace("tif", "png"), dpi=1000)
      plt.close(fig)
      print(dest.replace("tif", "png"))

      rgb[rgb == 1] = 0
      rgb[blue_mask, :] = blue_data
      print(str(f))

      rgb[np.isnan(rgb)] = 0

## Image Segmentation

In [27]:
def segmentation(img_path, img_back_path):
  #read two source images in grayscale
  img1 = gdal.Open(img_path).ReadAsArray()
  img2 = gdal.Open(img_back_path).ReadAsArray()

  #img1 = convert_to_RGB_range_T(img1, [0.005, 0.015, 0.015])
  img1 = np.transpose(img1, axes=(1, 2, 0))
  img1 = img1[:, :, :-1]
  img2 = np.transpose(img2, axes=(1, 2, 0))
  img2 = img2[:, :, :-1]


  img1 = cv2.merge([img1[:, :, 2], img1[:, :, 1], img1[:, :, 0]]) # RGB to BGR
  img2 = cv2.merge([img2[:, :, 2], img2[:, :, 1], img2[:, :, 0]]) # RGB to BGR
  img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
  img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  #plt.imshow(img1_gray, cmap='gray')

  
  #get the dimensions of the source  
  h, w = img2_gray.shape  
  # img1_gray = cv2.resize(img1_gray, (w, h)) 

  #create a zeroed out image of same dimensions  
  res1 = np.zeros((h,w,1), np.uint8)  

  #calculate what disappeared  
  disappeared = cv2.subtract(img1_gray, img2_gray)  
      
  #calculate what appeared  
  appeared = cv2.subtract(img2_gray, img1_gray)  

  #set threshold values
  thresh = 70
  maxValue = 255

  # apply gaussian blur and otsu's threshold  
  blur1 = cv2.GaussianBlur(disappeared,(5,5),0)  
  ret,disappear = cv2.threshold(blur1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)  
  blur2 = cv2.GaussianBlur(appeared,(5,5),0)  
  ret,appear = cv2.threshold(blur2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 

  # write result image  
  cv2.imwrite('disappeared.jpg', disappeared)  
  cv2.imwrite('appeared.jpg', appeared)  

  #img1 = cv2.cvtColor(appeared, cv2.COLOR_BGR2GRAY)
  #img2 = cv2.cvtColor(disappeared, cv2.COLOR_BGR2GRAY)
  img1 = appeared
  img2 = disappeared

  # create a CLAHE object (Arguments are optional).  
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))  
    
  cl1 = clahe.apply(img1)  
  #cv2.imwrite('clahe_A.jpg',cl1)  
    
  cl2 = clahe.apply(img2)  
  #cv2.imwrite('clahe_B.jpg',cl2)  

  threshold_area=100
  image_src = appeared
  im1 = cl1
  im2 = cl2
  
  # gray = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
  gray = image_src

  ret, gray = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)


  #image, contours, hierarchy = cv2.findContours(gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  contours, hierarchy = cv2.findContours(gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)


  mask = np.zeros(image_src.shape, np.uint8)
  cnts = sorted(contours, key=cv2.contourArea)

  for c in cnts:
    area=cv2.contourArea(c)
    if area > threshold_area:
      cv2.drawContours(mask, [c], -1, (255,255,255), -1)

  cv2.imwrite("mask.jpg", mask)

  return mask

## Key Point Detction with SIFT using Geo Information as threshold

In [28]:
# threshold for increase the contraction
CONTRACT_THRESHOLD = [248, 248, 245]

# find the threshold for RGB to high light only the first x% points
def find_RGB_threshold(raster, RGB_percentage):
    threshold = [0, 0, 0]
    for i in range(3):
        flat = raster[i].flatten()
        flat_len = flat.shape[0]
        num_element = floor(flat_len*(1-RGB_percentage[i]))
        flat = (flat*255).astype(int)
        bins = np.bincount(flat)

        for j in range(bins.shape[0]):
            num_element -= bins[j]
            if num_element <= 0:
                threshold[i] = (255-j)
                break
    return threshold

# remove -9999 to zero, normalize by spectrum, and convert to RGB,
# enhance the contraction for point under the threshold
def convert_to_RGB_range_T(raster, RGB_percentage):
    raster[raster < 0] = 0

    contract_threshold = find_RGB_threshold(raster, RGB_percentage)

    for i in range(raster.shape[0]):
        channel = raster[i]
        data_min = np.amin(channel)
        data_max = np.amax(channel)
        channel = (channel - data_min) / data_max

        channel = (1 - channel) * 255
        channel[channel < contract_threshold[i]] = 0
        raster[i] = channel
    return raster.astype(np.uint8)

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 1.0
TOP_MATCH_NUM = 200

# based on https://learnopencv.com/feature-based-image-alignment-using-opencv-c-python/
# find feature point using ORB or SIFT, SURF is not support by opencv due to patent issue in current version
def alignImages(im1, im2, transform_img, transform_back, detection_method, save_path, distance_threshold):
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    detector = cv2.ORB_create(MAX_FEATURES)

    # Detect ORB features and compute descriptors.
    if detection_method == 'SIFT':
        detector = cv2.SIFT_create()
        #detector = cv2.features2d.SIFT_create()
    # elif detection_method == 'SURF':
    #     detector = cv2.FastFeatureDetector_create()

    keypoints1, descriptors1 = detector.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = detector.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

    # if (detection_method == 'SIFT') | (detection_method == 'SURF'):
    #     matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_SL2)

    if detection_method == 'SIFT':
        matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_SL2)

    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    # the distance is not related to the real distance so disgard
    #matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    # numGoodMatches = int(min(TOP_MATCH_NUM, len(matches)))
    matches = matches[:numGoodMatches]

    # extrace good location
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    points1, points2, matches, points1_geo, points2_geo, distance_list = delete_far_matches(points1, points2, matches,
                                                                                            transform_img,
                                                                                            transform_back,
                                                                                            distance_threshold)

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite(save_path + ".jpg", imMatches)

    # Find homography
    # h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    # height, width, channels = im2.shape
    # im1Reg = cv2.warpPerspective(im1, h, (width, height))

    # return im1Reg, h, points1, points2, matches
    return points1, points2, matches, points1_geo, points2_geo, distance_list

def calc_avg_shift(points_1, points_2, transform_1, transform_2):
    xOrigin_1, yOrigin_1, pixelWidth_1, pixelHeight_1 = transform_1[0], transform_1[3], transform_1[1], transform_1[5]
    xOrigin_2, yOrigin_2, pixelWidth_2, pixelHeight_2 = transform_2[0], transform_2[3], transform_2[1], transform_2[5]
    
    distance_list = np.zeros((points_1.shape[0], 2), dtype=np.float32)
    for i in range(points_1.shape[0]):
        point_1, point_2 = points_1[i], points_2[i]
        point_1_x, point_1_y = floor(point_1[0]), floor(point_1[1])
        point_2_x, point_2_y = floor(point_2[0]), floor(point_2[1])
        
        geo_1_x = point_1_x * pixelWidth_1 + xOrigin_1
        geo_1_y = point_1_y * pixelHeight_1 + yOrigin_1
        
        geo_2_x = point_2_x * pixelWidth_2 + xOrigin_2
        geo_2_y = point_2_y * pixelHeight_2 + yOrigin_2
        
        distance_list[i, :] = geo_1_x - geo_2_x, geo_1_y - geo_2_y
    return distance_list

# detele the point that is too far in geographic distance by threshold, set by the user, default 30, 30
def delete_far_matches(points_1, points_2, matches, transform_1, transform_2, distance_threshold):
    xOrigin_1, yOrigin_1, pixelWidth_1, pixelHeight_1 = transform_1[0], transform_1[3], transform_1[1], transform_1[5]
    xOrigin_2, yOrigin_2, pixelWidth_2, pixelHeight_2 = transform_2[0], transform_2[3], transform_2[1], transform_2[5]
    print("xOrigin_1, yOrigin_1: ", xOrigin_1, yOrigin_1)
    print("xOrigin_2, yOrigin_2: ", xOrigin_2, yOrigin_2)
    far_idx_list = []
    points1_geo = np.zeros((len(matches), 2), dtype=np.float32)
    points2_geo = np.zeros((len(matches), 2), dtype=np.float32)
    distance_list = np.zeros((len(matches), 2), dtype=np.float32)

    for i in range(points_1.shape[0]):
        point_1, point_2 = points_1[i], points_2[i]
        point_1_x, point_1_y = floor(point_1[0]), floor(point_1[1])
        point_2_x, point_2_y = floor(point_2[0]), floor(point_2[1])

        geo_1_x = point_1_x * pixelWidth_1 + xOrigin_1
        geo_1_y = point_1_y * pixelHeight_1 + yOrigin_1

        geo_2_x = point_2_x * pixelWidth_2 + xOrigin_2
        geo_2_y = point_2_y * pixelHeight_2 + yOrigin_2

        distance = [geo_1_x - geo_2_x, geo_1_y - geo_2_y]

        points1_geo[i, :] = [geo_1_x, geo_1_y]
        points2_geo[i, :] = [geo_2_x, geo_2_y]
        distance_list[i, :] = distance

        if (abs(distance[0]) > distance_threshold[0]) | (abs(distance[1]) > distance_threshold[1]):
            far_idx_list.append(i)

    # print(far_idx_list)
    points_1 = np.delete(points_1, far_idx_list, 0)
    points_2 = np.delete(points_2, far_idx_list, 0)
    matches = np.delete(matches, far_idx_list, 0)

    points1_geo = np.delete(points1_geo, far_idx_list, 0)
    points2_geo = np.delete(points2_geo, far_idx_list, 0)
    distance_list = np.delete(distance_list, far_idx_list, 0)

    return points_1, points_2, matches, points1_geo, points2_geo, distance_list

# main method load raster np ndarray and find the feature point
def get_img_matches(img_path, img_back_path, mask, detection_method, IS_ADV, distance_threshold, RGB_percentage, img_save_path,
                    csv_save_path):
    # load the img data
    geo_img_data = gdal.Open(img_path)
    geo_back_data = gdal.Open(img_back_path)
    raster_img = geo_img_data.ReadAsArray()
    raster_back = geo_back_data.ReadAsArray()

    raster_img = convert_to_RGB_range_T(raster_img, RGB_percentage)
    raster_img_T = np.transpose(raster_img, axes=(1, 2, 0))

    # normalize to min-max range

    raster_back_T = np.transpose(raster_back, axes=(1, 2, 0))
    raster_back_T = raster_back_T[:, :, :-1]


    if IS_ADV:
      #bg_loc = cv2.imread(mask_path)
      mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
      raster_back_T = np.where(mask==0, 0, raster_back_T)
      cv2.imwrite('raster_back_T.jpg', raster_back_T) 

    transform_img = geo_img_data.GetGeoTransform()
    transform_back = geo_back_data.GetGeoTransform()

    # print(raster_img.shape, raster_img_T.shape, raster_back.shape, raster_back_T.shape)

    # Registered image will be resotred in imReg.
    # The estimated homography will be stored in h.
    # imReg, h, points_img, points_back, matches = alignImages(raster_img_T, raster_back_T, "result/" + save_path)
    points_img, points_back, matches, points1_geo, points2_geo, distance_list = alignImages(raster_img_T, raster_back_T,
                                                                                            transform_img,
                                                                                            transform_back,
                                                                                            detection_method,
                                                                                            img_save_path,
                                                                                            distance_threshold)

    save_to_csv(points_img, points_back, points1_geo, points2_geo, distance_list, csv_save_path)

    return points_img, points_back


# save feature point to csv file
def save_to_csv(points_1, points_2, points1_geo, points2_geo, distance_list, save_path):
    np_save = np.c_[points_1, points_2, points1_geo, points2_geo, distance_list]
    pd_save = pd.DataFrame(np_save)

    pd_column = ["pixel_x1", "pixel_y1", "pixel_x2", "pixel_y2", "geograph_x1", "geograph1_y1", "geograph_x2",
                 "geograph1_y2", "distance_x", "distance_y"]
    pd_save.columns = pd_column
    pd_save.to_csv(save_path + ".csv", index=False)

# Main Function

In [29]:
def main():
  """Need to change the path before run!!!"""
  PATH = "/content/gdrive/MyDrive/RA/Feature Point Detection Project/Jingtang_Ma/superglue/"
  HYPERSPECTRAL_ALL_PATH = PATH + "hyperspectral_all"
  HYPERSPECTRAL_RGB_PATH = PATH + "hyperspectral_rgb/"
  BACKGROUND_PATH = PATH + "background"
  RESULT_PATH = "/content/gdrive/MyDrive/RA/Feature Point Detection Project/Jingtang_Ma/superglue/results"

  detection_method = "SIFT"
  distance_threshold = [7.5, 7.5]
  RGB_percentage = [0.005, 0.015, 0.015]
  img_save_path = RESULT_PATH
  csv_save_path = RESULT_PATH
  
  IS_HYPERSPECTRAL_TO_RGB = False # Set True if needed
  IS_SEGMENTATION = True # Set True if needed
  IS_ADV = True # Set True if needed
  
  if IS_HYPERSPECTRAL_TO_RGB: hyperspectral2rgb(HYPERSPECTRAL_ALL_PATH)
  if IS_SEGMENTATION: mask = segmentation("/content/gdrive/MyDrive/RA/Feature Point Detection Project/Jingtang_Ma/superglue/background/google_bg.tif", "/content/gdrive/MyDrive/RA/Feature Point Detection Project/Jingtang_Ma/superglue/background/yandex_bg.tif")

  get_img_matches("/content/gdrive/MyDrive/RA/Feature Point Detection Project/Jingtang_Ma/superglue/hyperspectral_rgb/20210703_16_Reinfsteck_RGB.tif", "/content/gdrive/MyDrive/RA/Feature Point Detection Project/Jingtang_Ma/superglue/background/google_bg.tif", mask, detection_method, IS_ADV, distance_threshold, RGB_percentage, img_save_path,
                  csv_save_path)


main()

(931, 1551, 3)
(931, 1551) (931, 1551)
931 1551
(931, 1551) (931, 1551)
max:  255
min:  0
shape:  (931, 1551)
np.unique:  [  0 255]
xOrigin_1, yOrigin_1:  389510.0 4429435.0
xOrigin_2, yOrigin_2:  389510.0 4429435.0
